# Autograd: Automatic Differentiation

Central to all neural networks in PyTorch is the `autograd` package. Let’s first briefly visit this, and we will then go to training our first neural network.

The `autograd` package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

## Tensor

`torch.Tensor` is the central class of the package. If you set its attribute `.requires_grad` as `True`, it starts to track all operations on it. When you finish your computation you can call `.backward()` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into `.grad` attribute.

To stop a tensor from tracking history, you can call `.detach()` to detach it from the computation history, and to prevent future computation from being tracked.

To prevent tracking history (and using memory), you can also wrap the code block in `with torch.no_grad():`. This can be particularly helpful when evaluating a model because the model may have trainable parameters with `requires_grad=True`, but for which we don’t need the gradients.

There’s one more class which is very important for autograd implementation - a `Function`.

`Tensor` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a `.grad_fn` attribute that references a `Function` that has created the `Tensor` (except for Tensors created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a `Tensor`. If `Tensor` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to `backward()`, however if it has more elements, you need to specify a `gradient` argument that is a tensor of matching shape.

In [37]:
import torch

#### Create a tensor and set `requires_grad=True` to track computation with it

In [38]:
x = torch.ones(2, 2)
x.requires_grad  # Default `requires_grad` is False.

False

In [39]:
x = torch.ones(2, 2, requires_grad=True)
display(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

#### Do a tensor operation 

In [40]:
y = x + 2.0
display(y)  # Notice the grad_fn=<AddBackward0>

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

#### `y` was created as a result of an operation, so it has a `grad_fn`. 

In [41]:
y.grad_fn  # Shows AddBackward0 object and its memory location (not any kind of source code) - an Autograd object specific to an operation (here, Add). 

In [42]:
z = y * y * 3
out = z.mean()

display(z, out)  # Note the MulBackward0 and MeanBackward0 objects.

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)

tensor(27., grad_fn=<MeanBackward0>)

#### `.requires_grad_( ... )` changes an existing Tensor’s `requires_grad` flag in-place. The input flag defaults to `False` if not given. 

In [43]:
a = torch.randn(2, 2)
display(a)
a = ((a * 3) / (a - 1))
display(a)
print("a.requires_grad = {}".format(a.requires_grad))

tensor([[-0.1872, -0.3778],
        [ 0.9566, -0.0559]])

tensor([[  0.4731,   0.8227],
        [-66.0768,   0.1589]])

a.requires_grad = False


In [44]:
# Set `requires_grad` to `True` manually.
a.requires_grad_(True)  # Mutating operation in-place, so `_`
print("a.requires_grad = {}".format(a.requires_grad))

a.requires_grad = True


In [45]:
b = (a * a).sum()
display(b, b.grad_fn)

tensor(4367.0708, grad_fn=<SumBackward0>)

## Gradients (see [Autograd reference](https://pytorch.org/docs/stable/autograd.html))

#### Let’s backprop now. Because `out` contains a single scalar, `out.backward()` is equivalent to `out.backward(torch.tensor(1.))`. 

See [reference](https://pytorch.org/docs/stable/autograd.html#torch.Tensor) for details.

In [46]:
out

tensor(27., grad_fn=<MeanBackward0>)

In [47]:
out.backward(gradient=torch.tensor(1.))
# Which does: 
#  get gradients d_out/d_<Tensor> 
#  which are all the Tensor's involved - here: x, y, z
# Although for some reason `y.grad` and `z.grad` are `None`...

#### Print gradients d(out)/dx

In [48]:
# Remember:
# out = z.mean() <-- z = y * y * 3 <-- y = x + 2.0 <-- x = torch.ones(2, 2, requires_grad=True)
display(x.grad)
# y.grad, z.grad <-- These come out as None for some reason.

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

In [49]:
# BTW this will give an Exception:
# `out.backward()`  # Calling a second time.